In [4]:
%pip install langchain
%pip install python-dotenv
%pip install openai
%pip install tiktoken
%pip install pypdf
%pip install chromadb
%pip install fastapi nest-asyncio pyngrok uvicorn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Libraries
import os
import openai
import uvicorn
import nest_asyncio

from dotenv import load_dotenv, find_dotenv
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.llms import OpenAI
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from pyngrok import ngrok

data_path = "./data/"
data = os.listdir(data_path)

In [2]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [3]:
loader = PyPDFDirectoryLoader(data_path)
docs = loader.load()

In [4]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap  = 100
)

In [5]:
texts = text_splitter.split_documents(docs)

In [6]:
vectorstore = Chroma.from_documents(
  documents=texts,
  embedding=OpenAIEmbeddings(),
  persist_directory='./cache/'
)
vectorstore.persist()

In [7]:
print(texts[30])

page_content='adolescents, published between 1985 and 1995\n(Baer & Nietzel, 1991; Casey & Berman, 1985; Gross-\nman & Hughes, 1992; Hazelrigg, Cooper, & Borduin,\n1987; Kazdin, Siegel, & Bass, 1990; Shadish et al,\n1993; Weisz & Weiss, 1993; Weisz, Weiss, Alicke, &\nKlotz, 1987; Weisz, Weiss, Han, Granger, & Morton,\n1995). These reviews varied in the time period cov-\nered by the literature and in the age groups covered.\nThe most recent by Weisz and colleagues (1995) in-\ncludes studies through 1993. Casey and Berman\n(1985) examined outcome studies on children aged 12\nyears and younger while others included adolescents.\nThere were also some differences in the meta-analytic\napproach adopted by each group, but despite this,\nfindings are relatively congruent across the meta-\nanalyses conducted.\nThe major findings as summarized by Kazdin\n(p. 11, 1996) include:\n• The changes achieved with psychotherapy are\ngreater than those that emerge without treat-\nment. This finding has be

In [8]:
chat_history = []
chat = ConversationalRetrievalChain.from_llm(
    OpenAI(temperature=0),
    vectorstore.as_retriever()
)

In [9]:
def chatQuestion(question, chat_history):
    response = chat({"question": question,
                           "chat_history": chat_history})
    return response["answer"]

In [10]:
chatQuestion("What are the symptoms of depression?", chat_history)

' The symptoms of depression include a depressed mood most of the day, nearly every day, as indicated by either subjective report (e.g., feels sad or empty) or observation made by others (e.g., appears tearful); markedly diminished interest or pleasure in all, or almost all, activities most of the day, nearly every day; significant weight loss when not dieting or weight gain (e.g., a change of more than 5% of body weight in a month), or decrease or increase in appetite nearly every day; insomnia or hypersomnia nearly every day; psychomotor agitation or retardation nearly every day; fatigue or loss of energy nearly every day; feelings of worthlessness or excessive or inappropriate guilt (which may be delusional) nearly every day; and diminished ability to think or concentrate, or indecisiveness, nearly every day.'

In [11]:
app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

class Prompt(BaseModel):
    user_prompt : str

@app.post('/chat')
async def Post_prompt(prompt : Prompt):
    return {"response" : chatQuestion(prompt.user_prompt, chat_history)}

In [ ]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

t=2023-09-14T19:50:34-0400 lvl=warn msg="ngrok config file found at legacy location, move to XDG location" xdg_path=C:\\Users\\sofia\\AppData\\Local/ngrok/ngrok.yml legacy_path=C:\\Users\\sofia\\.ngrok2\\ngrok.yml
INFO:     Started server process [18900]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://19a6-2800-cd0-7205-b200-a10d-e8e0-7844-6bca.ngrok.io
INFO:     127.0.0.1:55224 - "OPTIONS /chat HTTP/1.1" 200 OK
INFO:     127.0.0.1:55224 - "POST /chat HTTP/1.1" 200 OK
INFO:     127.0.0.1:55265 - "OPTIONS /chat HTTP/1.1" 200 OK
INFO:     127.0.0.1:55265 - "POST /chat HTTP/1.1" 200 OK
INFO:     127.0.0.1:55363 - "OPTIONS /chat HTTP/1.1" 200 OK
INFO:     127.0.0.1:55363 - "POST /chat HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [18900]


t=2023-09-14T20:32:00-0400 lvl=eror msg="heartbeat timeout, terminating session" obj=tunnels.session obj=csess id=270847a1b30d clientid=c08ad110e20f3691185baab3e907d6c0
t=2023-09-14T20:32:00-0400 lvl=eror msg="session closed, starting reconnect loop" obj=tunnels.session obj=csess id=96a3fc317c6b err="session closed"
t=2023-09-14T20:43:43-0400 lvl=eror msg="heartbeat timeout, terminating session" obj=tunnels.session obj=csess id=0812a394652f clientid=c08ad110e20f3691185baab3e907d6c0
t=2023-09-14T20:43:43-0400 lvl=eror msg="session closed, starting reconnect loop" obj=tunnels.session obj=csess id=96a3fc317c6b err="session closed"
t=2023-09-14T21:06:34-0400 lvl=eror msg="heartbeat timeout, terminating session" obj=tunnels.session obj=csess id=e0c3501965af clientid=c08ad110e20f3691185baab3e907d6c0
t=2023-09-14T21:06:34-0400 lvl=eror msg="session closed, starting reconnect loop" obj=tunnels.session obj=csess id=96a3fc317c6b err="session closed"
t=2023-09-14T21:06:34-0400 lvl=eror msg="faile